# Make your requests faster

When you start scraping web-pages or requesting APIs, you will be facing a problem when doing a lot of requests: this is really slow!

It's because python is slow! You could say, well it should not be. Let's see how we can speed things up!

## Measure performances

In this notebook we will need to track how much time some code is taking to execute.
To make things easier, we will create a simple decorator that will print the number of micro-seconds a function takes to execute.

A good opportonity to practice decorators in a practical example!

*Note that you need python3.3 or higher.*

In [1]:
import time

def print_timing(func):
    '''Create a timing decorator function use @print_timing just above the function you want to time.'''

    def wrapper(*arg):
        start = time.perf_counter()
        
        # Run the function decorated
        result = func(*arg)

        end = time.perf_counter()
        execution_time = round((end - start), 2)
        print(f'{func.__name__} took {execution_time} sec')
        return result

    return wrapper


@print_timing
def example():
    time.sleep(2)


example()

example took 2.0 sec


## The API

For this example, we will use the [quotable.io](https://api.quotable.io) API. It's an online API you can use to generate random quote.

But feel free to replace `api_url` value with any API you'd like.

In [2]:
api_url = "https://api.quotable.io/random"

## The "classic" way

If you start playing with requests, your should probably have something like this:

In [3]:
import requests


def basic_request(url: str):
    response = requests.get(url)
    response_json = response.json()
    print(response_json["content"])


@print_timing
def basic_loop_request(url: str):
    # Query 50 times the API
    for _ in range(50):
        basic_request(url)


basic_loop_request(api_url)

Technology frightens me to death. It's designed by engineers to impress other engineers. And they always come with instruction booklets that are written by engineers for other engineers — which is why almost no technology ever works.
A hero is no braver than an ordinary man, but he is braver five minutes longer.
A really great talent finds its happiness in execution.
Maxim for life: You get treated in life the way you teach people to treat you.
Ignorant men raise questions that wise men answered a thousand years ago.
The more you like yourself, the less you are like anyone else, which makes you unique.
I have always thought the actions of men the best interpreters of their thoughts.
Life is so constructed that an event does not, cannot, will not, match the expectation.
Two persons cannot long be friends if they cannot forgive each other's little failings.
I know but one freedom and that is the freedom of the mind.
You can always count on Americans to do the right thing - after they've 

### Results

On my machine it took **17.06 sec for 50 requests**. 

Pretty slow right? But why is that?

Each time you make a request, your computer needs to create a new "session", format your request, send it and wait to receive the response before doing it again with the next request.

## The "session" way

To speed this, we can use a **"session"** that will be share by all the requests.

You can picture it as a postman that knows you already, so he knows which bell to ring, where is the mailbox,... Instead of having to search for those each time.

In [4]:
from requests import Session


def session_request(url: str, session: Session):
    # Instead of using request.get, we use our session
    response = session.get(url)
    response_json = response.json()
    print(response_json["content"])


@print_timing
def session_loop_request(url: str):
    # Create shared session for all of your requests
    with Session() as session:
        # Query 50 times the API
        for _ in range(50):
            session_request(url, session)


session_loop_request(api_url)

In the business world, the rearview mirror is always clearer than the windshield.
Effort only fully releases its reward after a person refuses to quit.
There are two kinds of failures: those who thought and never did, and those who did and never thought.
What the caterpillar calls the end of the world, the master calls a butterfly.
Whenever you have eliminated the impossible, whatever remains, however improbable, must be the truth.
Nothing could be worse than the fear that one had given up too soon and left one unexpended effort that might have saved the world.
Ideals are an imaginative understanding of that which is desirable in that which is possible.
The ego is nothing other than the focus of conscious attention.
In action a great heart is the chief qualification. In work, a great head.
Each man has his own vocation; his talent is his call. There is one direction in which all space is open to him.
Thought is the blossom; language the bud; action the fruit behind it.
You really can c

### Results

It took me **5.99 sec for 50 requests**. That's better!

And as you can see, I didn't change that much in the code.

## The "Async" way

If you need even more performances, you will need to use [AsyncIo](https://docs.python.org/3/library/asyncio.html).

This is a library to allow you to run asynchronous code.

Why is that more efficiant? Well, when you send a request you need to wait for the response. And during the waiting time, our computer does nothing.
If you count all the time the computer is just "waiting" on 50 or more requests, you will be surprised to see that most of the computing time is just waiting for the server to respond.

[AsyncIo](https://docs.python.org/3/library/asyncio.html) allow you to bypass that.

But as always, it has a cost: complexity.

Making your code async will complixify the code a lot and make the debugging not a pleasant experience. Also, you will go so fast that you could be banned by the server.

My advice? Use it only if you need it.

I will show you a simple example but you want to understand it better, I really advice you **[this video](https://www.youtube.com/watch?v=qAh5dDODJ5k)**!

### Requirements
In order to simplify a bit the code, I will use [httpx](https://www.python-httpx.org/) a python library that is working the same way as the `requests` module but with few helpers for async.

In [42]:
!pip install httpx

### Warning!
This code won't work in jupyter notebook, there are subtilities for async in jupyter notebook. See [this thread](https://stackoverflow.com/questions/47518874/how-do-i-run-python-asyncio-code-in-a-jupyter-notebook) for more informations.

To make it simpler, I will put this code in a .py file and run it in command line:

```python
from httpx import AsyncClient
import asyncio
import time


api_url = "https://api.quotable.io/random"


async def session_request_async(url: str, session: AsyncClient):
    # Instead of using request.get, we use our session
    response = await session.get(url)
    response_json = response.json()
    print(response_json["content"])
    return response_json


async def session_loop_request_async(url: str):
    # Create shared session for all of your requests
    async with AsyncClient() as session:
        # Create a list of empty tasks
        tasks = []
        # Query 50 times the API
        for _ in range(50):
            # Add a request to tasks
            tasks.append(
                asyncio.create_task(
                    session_request_async(url, session)        
                )
            )
        # Now that all the tasks are registred, run them
        responses = await asyncio.gather(*tasks)
            
            


start = time.perf_counter()

# We need to use asyncio.run to run the async function
asyncio.run(session_loop_request_async(api_url))

end = time.perf_counter()
execution_time = round((end - start), 2)
print(f'session_loop_request_async took {execution_time} sec')
```

In [5]:
!python3 ./assets/async_requests.py

It is impossible to love and to be wise.
Learning is the beginning of wealth. Learning is the beginning of health. Learning is the beginning of spirituality. Searching and learning is where the miracle process all begins.
Happiness does not come from doing easy work but from the afterglow of satisfaction that comes after the achievement of a difficult task that demanded our best.
The thing always happens that you really believe in; and the belief in a thing makes it happen.
Wisdom begins at the end.
To succeed, we must first believe that we can.
Wisdom begins in wonder.
Never bend your head. Always hold it high. Look the world right in the eye.
A good hockey player plays where the puck is. A great hockey player plays where the puck is going to be.
True friendship multiplies the good in life and divides its evils. Strive to have friends, for life without friends is like life on a desert island... to find one real friend in a lifetime is good fortune; to keep him is a blessing.
When the 

### Results
It only took me **0.8 sec for 50 requests**! That's impressive.

But as you can see, it is harder to write, structure and debug. So make sure you **really** need it if you consider using this method.

## Summary

If we gather all our results:

| Method                     | Execution time for 50 requests |
|----------------------------|--------------------------------|
| `requests.get` loop        | 17.06 sec                  |
| `requests` with `Session`  | 5.99 sec                   |
| `httpx` with `AsyncClient` | 0.8 sec                   |